In [7]:
import nltk
from nltk.corpus import stopwords
from string import punctuation
import pandas as pd
import csv


In [5]:


russian_stopwords = stopwords.words("russian")

text = "Ты не забывай, что у меня в голове опилки. Длинные слова меня только огорчают!"
tokens = nltk.word_tokenize(text.lower())
tokens = [token for token in tokens if token not in russian_stopwords and token not in punctuation]
tokens

['забывай', 'голове', 'опилки', 'длинные', 'слова', 'огорчают']

In [ ]:
dataset = pd.read_csv('Doctors1.csv', sep=';', error_bad_lines=False)


In [ ]:

# dataset.T
